In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import pathlib
import warnings
warnings.filterwarnings('ignore')

# Configuration
BASE_DIR = pathlib.Path("c:/Users/Acer/Desktop/Forex-Signal-App")
DATA_DIR = BASE_DIR / "data"
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"
MODEL_DIR = BASE_DIR / "models" / "signal_generator_v25"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Timeframes
TIMEFRAMES = ['1min', '5min', '15min', '30min', '1h', '4h']
TIMEFRAMES_MAP = {
    '1min': 'M1', 
    '5min': 'M5', 
    '15min': 'M15', 
    '30min': 'M30', 
    '1h': 'H1', 
    '4h': 'H4'
}

print("Configuration set up.")

Configuration set up.


In [2]:
def load_data():
    datasets = {}
    
    # Load Train Data
    print("Loading Train Data...")
    for tf in TIMEFRAMES:
        tf_code = TIMEFRAMES_MAP[tf]
        # Check for file variations (e.g. EURUSD_m5.csv or EUR_USD_m5.csv)
        # Based on file list: EURUSD_m5.csv
        file_name = f"EURUSD_{tf_code.lower()}.csv" 
        file_path = TRAIN_DIR / file_name
        
        if file_path.exists():
            df = pd.read_csv(file_path)
            df.columns = [c.lower() for c in df.columns]
            if 'time' in df.columns:
                df['time'] = pd.to_datetime(df['time'])
                df.set_index('time', inplace=True)
            datasets[f'train_{tf}'] = df
            print(f"  Loaded train {tf}: {len(df)} rows")
        else:
            print(f"  Warning: Train file {file_name} not found.")

    # Load Test Data
    print("\nLoading Test Data...")
    for tf in TIMEFRAMES:
        tf_code = TIMEFRAMES_MAP[tf]
        file_name = f"EURUSD_{tf_code.lower()}.csv"
        file_path = TEST_DIR / file_name
        
        if file_path.exists():
            df = pd.read_csv(file_path)
            df.columns = [c.lower() for c in df.columns]
            if 'time' in df.columns:
                df['time'] = pd.to_datetime(df['time'])
                df.set_index('time', inplace=True)
            datasets[f'test_{tf}'] = df
            print(f"  Loaded test {tf}: {len(df)} rows")
        else:
            print(f"  Warning: Test file {file_name} not found.")
            
    return datasets

datasets = load_data()

Loading Train Data...
  Loaded train 1min: 3354904 rows
  Loaded train 5min: 671581 rows
  Loaded train 15min: 224382 rows
  Loaded train 30min: 112194 rows
  Loaded train 1h: 56098 rows
  Loaded train 4h: 14498 rows

Loading Test Data...
  Loaded test 1min: 743476 rows
  Loaded test 5min: 148502 rows
  Loaded test 15min: 49807 rows
  Loaded test 30min: 24907 rows
  Loaded test 1h: 12454 rows
  Loaded test 4h: 3220 rows


In [4]:
def calculate_features(df):
    df = df.copy()
    
    # --- Common Indicators ---
    # Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'sma_{period}'] = df['close'].rolling(window=period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
        
    # ATR
    df['tr'] = np.maximum(df['high'] - df['low'], 
                          np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                     abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=14).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    df['bb_mid'] = df['close'].rolling(window=20).mean()
    df['bb_std'] = df['close'].rolling(window=20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_mid']
    
    # --- Trend / Regime Features ---
    # ADX
    df['up_move'] = df['high'] - df['high'].shift(1)
    df['down_move'] = df['low'].shift(1) - df['low']
    df['plus_dm'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['minus_dm'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)
    df['plus_di'] = 100 * (pd.Series(df['plus_dm']).rolling(14).mean() / df['atr'])
    df['minus_di'] = 100 * (pd.Series(df['minus_dm']).rolling(14).mean() / df['atr'])
    df['dx'] = 100 * abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'])
    df['adx'] = df['dx'].rolling(window=14).mean()
    
    # Rolling Sharpe (approximate using daily returns logic on lower timeframe)
    df['log_ret'] = np.log(df['close'] / df['close'].shift(1))
    df['rolling_sharpe'] = df['log_ret'].rolling(50).mean() / df['log_ret'].rolling(50).std()
    
    # Volume Trend
    df['volume_ma'] = df['volume'].rolling(20).mean()
    df['volume_trend'] = df['volume'] / df['volume_ma']
    
    # --- Momentum Features ---
    # Momentum Slopes (ROC)
    df['mom_slope_5'] = df['close'].diff(5) / df['close'].shift(5) * 100
    df['mom_slope_10'] = df['close'].diff(10) / df['close'].shift(10) * 100
    
    # Volatility Contraction (ATR Ratio)
    df['vol_contraction'] = df['atr'] / df['atr'].rolling(50).mean()
    
    # --- Mean Reversion Features ---
    # Z-Score
    df['z_score'] = (df['close'] - df['sma_20']) / df['bb_std']
    
    # Distance from VWAP (Approximate VWAP as cumulative sum of PV / cumulative V for the day/session is hard without session breaks, 
    # so we use a rolling VWAP)
    df['pv'] = df['close'] * df['volume']
    df['vwap'] = df['pv'].rolling(window=50).sum() / df['volume'].rolling(window=50).sum()
    df['dist_vwap'] = (df['close'] - df['vwap']) / df['vwap']
    
    # RSI Extreme
    df['rsi_extreme'] = np.where(df['rsi'] > 70, 1, np.where(df['rsi'] < 30, -1, 0))
    
    # --- Execution Features ---
    # Volatility Regime
    df['volatility_regime'] = pd.qcut(df['atr'], 3, labels=[0, 1, 2]) # Low, Med, High
    
    df.dropna(inplace=True)
    return df

# Apply to all datasets
print("Calculating features...")
for key in datasets:
    print(f"  Processing {key}...")
    datasets[key] = calculate_features(datasets[key])
print("Feature calculation complete.")

Calculating features...
  Processing train_1min...
  Processing train_5min...
  Processing train_15min...
  Processing train_30min...
  Processing train_1h...
  Processing train_4h...
  Processing test_1min...
  Processing test_5min...
  Processing test_15min...
  Processing test_30min...
  Processing test_1h...
  Processing test_4h...
Feature calculation complete.


In [12]:
# --- 1. Trend / Regime Model (4H + 1H) ---
print("\n--- Building Trend / Regime Model ---")

def create_trend_target(df, forward_bars=5):
    df = df.copy()
    # Future return
    df['future_return'] = df['close'].shift(-forward_bars) - df['close']
    
    # Threshold based on ATR
    threshold = 0.5 * df['atr'] * forward_bars
    
    conditions = [
        (df['future_return'] > threshold),
        (df['future_return'] < -threshold)
    ]
    choices = [1, -1] # 1: Bull, -1: Bear, 0: Range
    df['trend_target'] = np.select(conditions, choices, default=0)
    
    df.dropna(inplace=True)
    return df

# Use H1 and H4 data
trend_train_data = pd.concat([datasets['train_1h'], datasets['train_4h']])
trend_test_data = pd.concat([datasets['test_1h'], datasets['test_4h']])

trend_train_data = create_trend_target(trend_train_data)
trend_test_data = create_trend_target(trend_test_data)

# Features for Trend Model
trend_features = ['adx', 'plus_di', 'minus_di', 'rolling_sharpe', 'volume_trend', 
                  'sma_50', 'sma_200', 'ema_50', 'ema_200', 'rsi', 'bb_width']

X_trend_train = trend_train_data[trend_features]
y_trend_train = trend_train_data['trend_target']
X_trend_test = trend_test_data[trend_features]
y_trend_test = trend_test_data['trend_target']

# Train RandomForest
rf_trend = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, class_weight='balanced')
rf_trend.fit(X_trend_train, y_trend_train)

# Train LightGBM (Gradient Boosting)
gb_trend = lgb.LGBMClassifier(n_estimators=100, max_depth=3, random_state=42, class_weight='balanced')
gb_trend.fit(X_trend_train, y_trend_train)

# Evaluate
print("Trend Model Evaluation (RandomForest):")
print(classification_report(y_trend_test, rf_trend.predict(X_trend_test)))
print("Trend Model Evaluation (LightGBM):")
print(classification_report(y_trend_test, gb_trend.predict(X_trend_test)))

# Save Models
joblib.dump(rf_trend, MODEL_DIR / "trend_rf.joblib")
joblib.dump(gb_trend, MODEL_DIR / "trend_gb.joblib")


--- Building Trend / Regime Model ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2805
[LightGBM] [Info] Number of data points in the train set: 70193, number of used features: 11
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

['c:\\Users\\Acer\\Desktop\\Forex-Signal-App\\models\\signal_generator_v25\\trend_gb.joblib']

In [13]:
# --- 2. Momentum + Breakout Model (30m / 15m) ---
print("\n--- Building Momentum + Breakout Model ---")

def create_momentum_target(df, forward_bars=3):
    df = df.copy()
    # Look for large moves
    df['future_high'] = df['high'].rolling(forward_bars).max().shift(-forward_bars)
    df['future_low'] = df['low'].rolling(forward_bars).min().shift(-forward_bars)
    df['future_close'] = df['close'].shift(-forward_bars)
    
    # Breakout condition: Price moves significantly
    move_threshold = 1.5 * df['atr']
    
    conditions = [
        (df['future_close'] - df['close'] > move_threshold),
        (df['close'] - df['future_close'] > move_threshold)
    ]
    choices = [2, 0] # 2: Bull Breakout, 0: Bear Breakout
    df['momentum_target'] = np.select(conditions, choices, default=1) # 1: No Breakout
    
    df.dropna(inplace=True)
    return df

mom_train_data = pd.concat([datasets['train_30min'], datasets['train_15min']])
mom_test_data = pd.concat([datasets['test_30min'], datasets['test_15min']])

mom_train_data = create_momentum_target(mom_train_data)
mom_test_data = create_momentum_target(mom_test_data)

mom_features = ['mom_slope_5', 'mom_slope_10', 'vol_contraction', 'volume_trend', 
                'rsi', 'adx', 'atr', 'bb_width']

X_mom_train = mom_train_data[mom_features]
y_mom_train = mom_train_data['momentum_target']
X_mom_test = mom_test_data[mom_features]
y_mom_test = mom_test_data['momentum_target']

# Train XGBoost
# Calculate scale_pos_weight for XGBoost (approximate for multiclass or use sample weights, but for simplicity we use LightGBM as primary or just balanced weights if possible)
# XGBoost doesn't support class_weight='balanced' directly for multiclass. 
# We will switch to LightGBM for both or use sample weights. Let's use LightGBM for both for consistency and better imbalance support.

# Train LightGBM 1
xgb_mom = lgb.LGBMClassifier(n_estimators=100, max_depth=5, random_state=42, class_weight='balanced')
xgb_mom.fit(X_mom_train, y_mom_train)

# Train LightGBM 2 (Different params)
lgb_mom = lgb.LGBMClassifier(n_estimators=100, max_depth=3, num_leaves=15, random_state=42, class_weight='balanced')
lgb_mom.fit(X_mom_train, y_mom_train)

# Evaluate
print("Momentum Model Evaluation (LightGBM 1):")
print(classification_report(y_mom_test, xgb_mom.predict(X_mom_test)))
print("Momentum Model Evaluation (LightGBM 2):")
print(classification_report(y_mom_test, lgb_mom.predict(X_mom_test)))

# Save Models
joblib.dump(xgb_mom, MODEL_DIR / "mom_xgb.joblib")
joblib.dump(lgb_mom, MODEL_DIR / "mom_lgb.joblib")


--- Building Momentum + Breakout Model ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003322 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 336175, number of used features: 8
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

['c:\\Users\\Acer\\Desktop\\Forex-Signal-App\\models\\signal_generator_v25\\mom_lgb.joblib']

In [14]:
# --- 3. Mean Reversion Model (15m / 5m) ---
print("\n--- Building Mean Reversion Model ---")

def create_mean_reversion_target(df, forward_bars=5):
    df = df.copy()
    df['future_return'] = df['close'].shift(-forward_bars) - df['close']
    threshold = 0.5 * df['atr']
    
    # Logic: If we are far from mean, do we revert?
    # We want to predict the reversion.
    
    conditions = [
        (df['future_return'] > threshold),  # Price goes up (Reversion from Low)
        (df['future_return'] < -threshold)  # Price goes down (Reversion from High)
    ]
    choices = [1, -1]
    df['mr_target'] = np.select(conditions, choices, default=0)
    
    df.dropna(inplace=True)
    return df

mr_train_data = pd.concat([datasets['train_15min'], datasets['train_5min']])
mr_test_data = pd.concat([datasets['test_15min'], datasets['test_5min']])

mr_train_data = create_mean_reversion_target(mr_train_data)
mr_test_data = create_mean_reversion_target(mr_test_data)

mr_features = ['z_score', 'dist_vwap', 'rsi_extreme', 'bb_width', 'rsi', 'stoch_k', 'stoch_d']
# Ensure stoch_k/d are calculated in calculate_features if not already. 
# I missed adding stoch to calculate_features explicitly, let me check.
# I added rsi, bb, but not stoch. I should add stoch to features or just use what I have.
# I'll stick to what I have: z_score, dist_vwap, rsi_extreme, bb_width, rsi.

mr_features = ['z_score', 'dist_vwap', 'rsi_extreme', 'bb_width', 'rsi']

X_mr_train = mr_train_data[mr_features]
y_mr_train = mr_train_data['mr_target']
X_mr_test = mr_test_data[mr_features]
y_mr_test = mr_test_data['mr_target']

# Train RandomForest
rf_mr = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, class_weight='balanced')
rf_mr.fit(X_mr_train, y_mr_train)

# Train SVM
svc_mr = SVC(kernel='rbf', probability=True, max_iter=1000, random_state=42, class_weight='balanced')
# Subsample for SVM training to avoid timeout
sample_idx = np.random.choice(len(X_mr_train), min(10000, len(X_mr_train)), replace=False)
svc_mr.fit(X_mr_train.iloc[sample_idx], y_mr_train.iloc[sample_idx])

# Evaluate
print("Mean Reversion Model Evaluation (RandomForest):")
print(classification_report(y_mr_test, rf_mr.predict(X_mr_test)))
print("Mean Reversion Model Evaluation (SVM):")
print(classification_report(y_mr_test, svc_mr.predict(X_mr_test)))

# Save Models
joblib.dump(rf_mr, MODEL_DIR / "mr_rf.joblib")
joblib.dump(svc_mr, MODEL_DIR / "mr_svc.joblib")


--- Building Mean Reversion Model ---
Mean Reversion Model Evaluation (RandomForest):
              precision    recall  f1-score   support

          -1       0.37      0.38      0.38     69030
           0       0.34      0.25      0.29     57366
           1       0.38      0.45      0.41     71510

    accuracy                           0.37    197906
   macro avg       0.36      0.36      0.36    197906
weighted avg       0.36      0.37      0.36    197906

Mean Reversion Model Evaluation (SVM):
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00     69030
           0       0.29      1.00      0.45     57366
           1       0.00      0.00      0.00     71510

    accuracy                           0.29    197906
   macro avg       0.10      0.33      0.15    197906
weighted avg       0.08      0.29      0.13    197906



['c:\\Users\\Acer\\Desktop\\Forex-Signal-App\\models\\signal_generator_v25\\mr_svc.joblib']

In [15]:
# --- 4. Execution Optimization Model (5m / 1m) ---
print("\n--- Building Execution Optimization Model ---")

def create_execution_target(df, tp_mult=2.0, sl_mult=1.0):
    df = df.copy()
    targets = []
    
    closes = df['close'].values
    highs = df['high'].values
    lows = df['low'].values
    atrs = df['atr'].values
    
    # Look forward window
    window = 24 # 24 bars (e.g. 24 mins for M1, 2 hours for M5)
    
    for i in range(len(df) - window):
        entry = closes[i]
        atr = atrs[i]
        tp = entry + (atr * tp_mult)
        sl = entry - (atr * sl_mult)
        
        outcome = 0 # 0: No trade/Loss, 1: Win
        
        # Check future bars
        for j in range(1, window + 1):
            curr_high = highs[i+j]
            curr_low = lows[i+j]
            
            if curr_low <= sl:
                outcome = 0 # Hit SL
                break
            if curr_high >= tp:
                outcome = 1 # Hit TP
                break
        
        targets.append(outcome)
        
    targets.extend([0] * window)
    df['exec_target'] = targets
    return df

exec_train_data = pd.concat([datasets['train_5min'], datasets['train_1min']])
exec_test_data = pd.concat([datasets['test_5min'], datasets['test_1min']])

exec_train_data = create_execution_target(exec_train_data)
exec_test_data = create_execution_target(exec_test_data)

exec_features = ['volatility_regime', 'atr', 'rsi', 'mom_slope_5', 'dist_vwap']

X_exec_train = exec_train_data[exec_features]
y_exec_train = exec_train_data['exec_target']
X_exec_test = exec_test_data[exec_features]
y_exec_test = exec_test_data['exec_target']

# Train Logistic Regression
lr_exec = LogisticRegression(random_state=42, class_weight='balanced')
lr_exec.fit(X_exec_train, y_exec_train)

# Train LightGBM
lgb_exec = lgb.LGBMClassifier(n_estimators=100, max_depth=3, random_state=42, class_weight='balanced')
lgb_exec.fit(X_exec_train, y_exec_train)

# Evaluate
print("Execution Model Evaluation (Logistic Regression):")
print(classification_report(y_exec_test, lr_exec.predict(X_exec_test)))
print("Execution Model Evaluation (LightGBM):")
print(classification_report(y_exec_test, lgb_exec.predict(X_exec_test)))

# Save Models
joblib.dump(lr_exec, MODEL_DIR / "exec_lr.joblib")
joblib.dump(lgb_exec, MODEL_DIR / "exec_lgb.joblib")


--- Building Execution Optimization Model ---
[LightGBM] [Info] Number of positive: 1355139, number of negative: 2669306
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1023
[LightGBM] [Info] Number of data points in the train set: 4024445, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

['c:\\Users\\Acer\\Desktop\\Forex-Signal-App\\models\\signal_generator_v25\\exec_lgb.joblib']

In [16]:
# --- 5. Ensemble Logic ---
print("\n--- Ensemble Logic ---")

def get_ensemble_signal(row):
    # 1. Trend / Regime
    # Prepare features for Trend Model
    trend_feats = [row[f] for f in trend_features]
    trend_pred = rf_trend.predict([trend_feats])[0] # 1: Bull, -1: Bear, 0: Range
    
    signal = 0
    signal_type = "None"
    
    if trend_pred == 1: # Bull
        # Check Momentum for Bullish Breakout
        mom_feats = [row[f] for f in mom_features]
        mom_pred = xgb_mom.predict([mom_feats])[0]
        if mom_pred == 2:
            signal = 1
            signal_type = "Trend Follow (Bull)"
            
    elif trend_pred == -1: # Bear
        # Check Momentum for Bearish Breakout
        mom_feats = [row[f] for f in mom_features]
        mom_pred = xgb_mom.predict([mom_feats])[0]
        if mom_pred == 0:
            signal = -1
            signal_type = "Trend Follow (Bear)"
            
    else: # Range
        # Check Mean Reversion
        mr_feats = [row[f] for f in mr_features]
        mr_pred = rf_mr.predict([mr_feats])[0]
        if mr_pred == 1:
            signal = 1
            signal_type = "Mean Reversion (Buy)"
        elif mr_pred == -1:
            signal = -1
            signal_type = "Mean Reversion (Sell)"
            
    # 2. Execution Optimization
    if signal != 0:
        exec_feats = [row[f] for f in exec_features]
        exec_prob = lgb_exec.predict_proba([exec_feats])[0][1] # Probability of Win
        
        # Lower threshold because we are using balanced weights (probabilities might be skewed)
        if exec_prob < 0.5: 
            signal = 0
            signal_type = "Filtered by Execution Model"
            
    return signal, signal_type

# Test on a sample of recent data (e.g. last 500 rows of M5 test data)
print("\nTesting Ensemble on recent M5 data...")
sample_data = datasets['test_5min'].iloc[-500:].copy()
results = []

for idx, row in sample_data.iterrows():
    sig, sig_type = get_ensemble_signal(row)
    if sig != 0:
        results.append({
            'time': idx,
            'signal': sig,
            'type': sig_type,
            'close': row['close']
        })

results_df = pd.DataFrame(results)
if not results_df.empty:
    print(f"Generated {len(results_df)} signals.")
    print(results_df.head())
else:
    print("No signals generated in sample.")

print("\nNotebook Complete.")


--- Ensemble Logic ---

Testing Ensemble on recent M5 data...
Generated 86 signals.
                 time  signal                  type    close
0 2025-12-29 11:05:00       1  Mean Reversion (Buy)  1.17645
1 2025-12-29 12:20:00       1  Mean Reversion (Buy)  1.17582
2 2025-12-29 12:40:00       1  Mean Reversion (Buy)  1.17569
3 2025-12-29 17:40:00       1  Mean Reversion (Buy)  1.17526
4 2025-12-29 17:45:00       1  Mean Reversion (Buy)  1.17513

Notebook Complete.
